## Create the DeployInst for advanced reactors

In [1]:
import numpy as np
import xmltodict
import sys
import matplotlib.pyplot as plt
import math
import pandas as pd

sys.path.insert(0, "../../../scripts")
import create_AR_DeployInst as cdi
import predicting_the_past_import as import_data

In [2]:
duration = 1500
reactor_prototypes = {'Xe-100':(76, 720), 'MMR':(5,240), 'VOYGR':(73, 720)}
scenarios = [['Xe-100'], ['MMR'], ['Xe-100','MMR'], ['MMR','VOYGR'], ['Xe-100','VOYGR'], ['Xe-100','MMR','VOYGR']]
for growth in ['nogrowth', '1percent']:
    for index, reactors in enumerate(scenarios):
        demand_equation = np.zeros(duration)
        if growth == 'nogrowth':
            demand_equation[721:] = 87198.156
        else:
            for ii in np.linspace(721, 1499, 779):
                demand_equation[int(ii)] = 87198.156*np.exp(0.00083*(int(ii)-721))
        scenario = '_'.join(scenarios[index]).replace('-','')
        scenario = scenario.lower() + "_" + growth 
        reactor_dict = {k: reactor_prototypes[k] for k in (reactors)}
        
        deployinst = cdi.convert_xml_to_dict("united_states/buildtimes/UNITED_STATES_OF_AMERICA/deployinst.xml")
        #lwr_powers = cdi.get_powers("../../../database/")
        
        lwr_powers = {}
        reactors = pd.read_csv("../../../database/reactors_pris_2020.csv")
        reactors = reactors.loc[reactors['Country'] == "UNITED STATES OF AMERICA"]
        for index, row in reactors.iterrows():
            lwr_powers[row['Unit']] = row['RUP [MWe]']
        lwr_powers = {k.replace(' ', '_'): v for k, v in lwr_powers.items()}
            
            
        deployed_lwr_dict = cdi.get_deployinst_dict(deployinst, lwr_powers, "united_states/reactors/")
        time, deployed_power = cdi.get_deployed_power(lwr_powers, deployed_lwr_dict, duration)
        power_gap = cdi.determine_power_gap(deployed_power*0.9266, demand_equation)
        
        deploy_schedule = cdi.determine_deployment_schedule(power_gap, reactor_dict)
        cdi.write_deployinst(deploy_schedule, "./united_states/buildtimes/" + scenario + "_deployinst.xml")